In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from vega_datasets import data
import psycopg2
#import textatistic
import seaborn as sbn
from altair import Chart, X, Y, Color, Scale
import altair as alt
from vega_datasets import data
import requests
from bs4 import BeautifulSoup
matplotlib.style.use('ggplot')
import os
import numpy as np

Tesfatsion Shiferaw

# Data source

I used the COVID-19 data and filtered only to include the data from China.
So, I generally collected my data from three sources:
1. https://www.kaggle.com/wang749/china2019ncov#Wuhan-2019-nCoV.csv - Covid 19 data
2. https://data.world/agriculture/china-agro-econ-data/workspace/project-summary?agentid=agriculture&datasetid=china-agro-econ-data - china population data by provinces
3. Wikipedia - I got life expectancy data by provinces from Wikipedia

# My question

I was interested in looking at the spread and effect that covid-19 had in specific provinces of china. The different provinces of china have different living standards. So, I believe that this will give us a sense of how the effect of covid-19 will be in different countries of the workd.
My question - Is there any specific relationship between life expectancy and the death rate/spread of covid-19?

# Steps

* clean up the data - the covid-19 data was arranged by date, so I had to sort it by provinces. I basically changed the number of rows from thousands to 30.
* I gathered the data from three different sources, so I had to merge them accordingly.
* Do some calculation for percentage mortality and number of infected. Then create a new colomn for them.
* draw graphs to answer my questions.

In [2]:
os.getcwd()

'C:\\Users\\tesfa'

In [3]:
covid = pd.read_csv("\\Users\\tesfa\\covid_19_data.csv") #Import covid-19 dataset

In [4]:
covid_china = covid[covid["Country/Region"] == 'Mainland China'] #Filter for data from china(csv)
covid_by_province = covid_china.groupby(['Province/State'], as_index = False) ['Confirmed', 'Deaths', 'Recovered'].sum()
covid_by_province.head()

,Province/State,Confirmed,Deaths,Recovered
0,Anhui,62497.0,341.0,46449.0
1,Beijing,29634.0,402.0,17315.0
2,Chongqing,37339.0,347.0,26517.0
3,Fujian,20013.0,48.0,13805.0
4,Gansu,7338.0,119.0,5286.0


In [6]:
china_prov = pd.read_csv("\\Users\\tesfa\\china_provinces_population.csv") #import china population data (csv)
china_prov.head(5)

,PROVINCE NAME,POPULATION
0,Anhui,62200728
1,Beijing,25215113
2,Chongqing,31791850
3,Fujian,39789666
4,Gansu,26406335


In [7]:
covid_by_province.columns

Index(['Province/State', 'Confirmed', 'Deaths', 'Recovered'], dtype='object')

In [8]:
china_prov.columns

Index(['PROVINCE NAME', 'POPULATION'], dtype='object')

In [9]:
#merge the first two datasets
new_df = pd.merge(covid_by_province, china_prov, left_on = 'Province/State', right_on = 'PROVINCE NAME')
new_df.head()

,Province/State,Confirmed,Deaths,Recovered,PROVINCE NAME,POPULATION
0,Anhui,62497.0,341.0,46449.0,Anhui,62200728
1,Beijing,29634.0,402.0,17315.0,Beijing,25215113
2,Chongqing,37339.0,347.0,26517.0,Chongqing,31791850
3,Fujian,20013.0,48.0,13805.0,Fujian,39789666
4,Gansu,7338.0,119.0,5286.0,Gansu,26406335


In [10]:
#Provinces Vs Population
alt.Chart(new_df).mark_bar().encode(
    alt.X('Province/State'),
    alt.Y('POPULATION')).properties(width=700, height=300)

alt.Chart(...)

In [12]:
#calculate no. of infected indivituals for every hundred of the population and add a new colomn
new_df['infected%pop'] = new_df['Confirmed']/new_df['POPULATION'] * 100
new_df.head()

,Province/State,Confirmed,Deaths,Recovered,PROVINCE NAME,POPULATION,infected%pop
0,Anhui,62497.0,341.0,46449.0,Anhui,62200728,0.100476
1,Beijing,29634.0,402.0,17315.0,Beijing,25215113,0.117525
2,Chongqing,37339.0,347.0,26517.0,Chongqing,31791850,0.117448
3,Fujian,20013.0,48.0,13805.0,Fujian,39789666,0.050297
4,Gansu,7338.0,119.0,5286.0,Gansu,26406335,0.027789


In [13]:
#calculate no. of death for every hundred of the population and add a new colomn
new_df['death%conf'] = new_df['Deaths']/new_df['Confirmed'] * 100
new_df.head()

,Province/State,Confirmed,Deaths,Recovered,PROVINCE NAME,POPULATION,infected%pop,death%conf
0,Anhui,62497.0,341.0,46449.0,Anhui,62200728,0.100476,0.545626
1,Beijing,29634.0,402.0,17315.0,Beijing,25215113,0.117525,1.356550
2,Chongqing,37339.0,347.0,26517.0,Chongqing,31791850,0.117448,0.929323
3,Fujian,20013.0,48.0,13805.0,Fujian,39789666,0.050297,0.239844
4,Gansu,7338.0,119.0,5286.0,Gansu,26406335,0.027789,1.621695


In [32]:
# Percentage of infected people in different provinces
# The heighest percent of infected population is in Hubie which capital city is Wuhan.
alt.Chart(new_df).mark_line().encode(
    alt.X('POPULATION'),
    alt.Y('infected%pop')).properties(width=700, height=300)

alt.Chart(...)

# Conclussion 1

Until the time when this data was collected the virus has reached out to roughly equal percent (0% - 1%) of population in each state except the province which has 60,000,000 residents (Hubei).  This doesn't mean that there is not a relationship between the number of infected people and the total population. It is fact that the more the population is, more people get infected. However, the ratio of people infected to total population stays the same. At least for now.

In [20]:
#This is the data that I got from wikipedia about the life expectancy for each province.
#I got the data in the form of list, so I had to conver to array(series) and convert it to dataframe.

arr = np.array([['Macau', 84.5], ['Zhejiang',81.43], ['Shanghai', 81.03], ['Hong Kong', 81.02], ['Beijing', 81.01],
                ['Tianjin', 78.89], ['Hainan', 78.87], ['Jiangsu', 78.84], ['Guangdong', 76.49], ['Shandong', 76.46],
               ['Liaoning', 76.38], ['Jilin', 76.18], ['Heilongjiang', 75.98], ['Fujian', 75.76], ['Chongqing', 75.7], 
                ['Guangxi', 75.11], ['Anhui', 75.08], ['Hebei',74.97], ['Shaanxi', 74.92], ['Jiangxi', 74.92], ['Hubei', 74.87],
               ['Sichuan', 74.75], ['Hunan', 74.7], ['Shanxi', 74.68], ['Henan',74.57], ['Inner Mongolia', 74.44],
               ['Ningxia', 73.38], ['Xinjiang', 72.35], ['Gansu', 72.23], ['Guizhou', 71.1], ['Qinghai', 69.96], 
               ['Yunnan', 69.54], ['Tibel', 64.37]])
life_expectancy = pd.DataFrame(arr, columns = ['Provinces', 'Life Expectancy'])
life_expectancy.head()

,Provinces,Life Expectancy
0,Macau,84.5
1,Zhejiang,81.43
2,Shanghai,81.03
3,Hong Kong,81.02
4,Beijing,81.01


In [16]:
#I merged it to the main data frame.
new_df = pd.merge(new_df, life_expectancy, left_on = 'Province/State', right_on = 'Provinces')
new_df.head()

,Province/State,Confirmed,Deaths,Recovered,PROVINCE NAME,POPULATION,infected%pop,death%conf,Provinces,Life Expectancy
0,Anhui,62497.0,341.0,46449.0,Anhui,62200728,0.100476,0.545626,Anhui,75.08
1,Beijing,29634.0,402.0,17315.0,Beijing,25215113,0.117525,1.356550,Beijing,81.01
2,Chongqing,37339.0,347.0,26517.0,Chongqing,31791850,0.117448,0.929323,Chongqing,75.7
3,Fujian,20013.0,48.0,13805.0,Fujian,39789666,0.050297,0.239844,Fujian,75.76
4,Gansu,7338.0,119.0,5286.0,Gansu,26406335,0.027789,1.621695,Gansu,72.23


* The following graph shows the relationship between life expectancy and mortality rate in various providences of China.

In [26]:
alt.Chart(new_df).mark_bar().encode(
    alt.X('Life Expectancy'),
    alt.Y('death%conf'), color = 'Province/State', tooltip = 'Province/State').properties(width=700, height=300)

alt.Chart(...)

## Conclusion 2

In the above graph, we don't see a regular smooth pattern. However, if we look in detail, we see more shorter bars to the right of the graph. Which means, the higher the life expectancy, the lower the mortality rate is. Moreover, the irregularity in the graph can be a clue that other factors also determine the morality rate of the disease. It needs more investigation.

In [31]:
# This graph too shows the same idea but in a different way.
alt.Chart(new_df).mark_circle().encode(
    alt.X('Province/State', bin=False),
    alt.Y('death%conf', bin=True),
    size='Life Expectancy',
    #color = 'POPULATION'
    
).properties(width=700, height=500)

alt.Chart(...)